In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from glob import glob
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image,display
import seaborn as sns
import matplotlib.image as mpimg
import scipy.spatial.distance as dist
from sklearn.model_selection import train_test_split
from skimage.measure import compare_ssim
import os

#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tr = pd.read_csv('/kaggle/input/landmark-recognition-2020/train.csv')
tr.head()

In [ ]:
print(tr.landmark_id.nunique())
print(np.max(tr.landmark_id))
print(np.min(tr.landmark_id))

In [ ]:
tab = tr.landmark_id.value_counts()
print(tab.head(5))
print(tab.tail(5))

In [ ]:
tr[tr.landmark_id==197219]

In [ ]:
mainPath = '/kaggle/input/landmark-recognition-2020/train/'
all_img_paths = [y for x in os.walk(mainPath) for y in glob(os.path.join(x[0], '*.jpg'))]
ids = [y for x in os.walk(mainPath) for y in glob(os.path.join(x[0]))]
all_filenames = []
for filepath in all_img_paths:
    FileName = os.path.basename(filepath)
    all_filenames.append(FileName)
path_dict = dict(zip(all_filenames,all_img_paths))

In [ ]:
df=pd.DataFrame()
df['fname'] = all_filenames
df['pname'] = all_img_paths
df['id'] = list(map(lambda x: x[:-4], df.fname))
df.head()

In [ ]:
print(df.shape)
train = pd.merge(tr, df, on = 'id')
train.head()

In [ ]:
mainPath = '/kaggle/input/landmark-recognition-2020/test/'
all_img_paths = [y for x in os.walk(mainPath) for y in glob(os.path.join(x[0], '*.jpg'))]
ids = [y for x in os.walk(mainPath) for y in glob(os.path.join(x[0]))]
all_filenames = []
for filepath in all_img_paths:
    FileName = os.path.basename(filepath)
    all_filenames.append(FileName)
path_dict = dict(zip(all_filenames,all_img_paths))

test=pd.DataFrame()
test['fname'] = all_filenames
test['pname'] = all_img_paths
print(test.head())

test['id'] = list(map(lambda x: x[:-4], test.fname))
print(test.head())

In [ ]:
test.shape

Have a look of some images from train and test data

In [ ]:
from PIL import Image
image = Image.open(train.pname[0])
print(image.size)
display(image)

In [ ]:
image = Image.open(train.pname[100])
print(image.size)
display(image)

In [ ]:
image = Image.open(test.pname[0])
print(image.size)
display(image)

In [ ]:
image = Image.open(test.pname[100])
print(image.size)
display(image)

In [ ]:
#os.listdir('../input/siim-isic-melanoma-classification/jpeg/train/')
def imtocsv(data):
    from PIL import Image
    nrow = []
    ncol = []
    pix = []
    for j in data.pname:
        image = Image.open(j)
        nrow.append(image.size[0])
        ncol.append(image.size[1])
        pix.append(image.size[0]*image.size[1])
    out = {'nrow': nrow, 'ncol': ncol, 'pix':pix}
    df= pd.DataFrame(out)
    df.insert(0,'id',data.id,True)
    return df

imd=imtocsv(train.iloc[0:4,:],5)
train_50 = pd.merge(train.iloc[0:4,:], imd, on = 'id')
train_50.head()

In [ ]:
out=imtocsv(train)
train_50 = pd.merge(train, out, on = 'id')
train_50.head()

In [ ]:
del out
out=imtocsv(test)
test_50 = pd.merge(test, out, on = 'id')
test_50.head()

In [ ]:
train_50.to_csv('train_summary.csv', index=False)
test_50.to_csv('test_summary.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.hist(train_50.pix) # add color='green',bins=20
plt.xlabel('Pixels')
plt.ylabel('Freq')
plt.title('Pixel value distribution in train data')
plt.show()

In [ ]:
plt.hist(test_50.pix) # add color='green',bins=20
plt.xlabel('Pixels')
plt.ylabel('Freq')
plt.title('Pixel value distribution in test data')
plt.show()

landmark_id == 138982 has maximum count in train data. Let's have a look at the pixel value distribution for this landmark.

In [ ]:
plt.hist(train_50.pix[train_50.landmark_id == 138982]) # add color='green',bins=20
plt.xlabel('Pixels')
plt.ylabel('Freq')
plt.title('Pixel value distribution in train data with landmark id 138982')
plt.show()

### High and low resolution images in train and test sets

In [ ]:
print('Maximum number of pixels: ', train_50.pix.max())
print('Minimum number of pixels: ', train_50.pix.min())
print('Maximum number of pixels: ', test_50.pix.max())
print('Minimum number of pixels: ', test_50.pix.min())

Let's have a look of images with high and low resolution.

In [ ]:
tmp = train_50.loc[train_50.pix == train_50.pix.max()]
tmp.head()

In [ ]:
pname = list(tmp.pname)
len(pname)

7592 Images with high resolution in Train data

### Lets go through some imaages in train data with high resolution

In [ ]:
image = Image.open(pname[0])
print(image.size)
display(image)

In [ ]:
image = Image.open(pname[1])
print(image.size)
display(image)

In [ ]:
image = Image.open(pname[2])
print(image.size)
display(image)

In [ ]:
image = Image.open(pname[7590])
print(image.size)
display(image)

In [ ]:
image = Image.open(pname[7591])
print(image.size)
display(image)

### Looks like there might be some relationship between train images with high resolution.
Let's display multiple images together

In [ ]:
fig=plt.figure(figsize=(12, 10))
columns = 5
rows = 5
for i in range(1, columns*rows +1):
    img = Image.open(pname[i])
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

### Lets go through some imaages in train data with low resolution

In [ ]:
tmp = train_50.loc[train_50.pix == train_50.pix.min()]
tmp.head()

Only one image with low resolution. 

In [ ]:
pname = list(tmp.pname)
print(len(pname))
image = Image.open(pname[0])
print(image.size)
display(image)

This is the image in training data with low resolation.

### Lets go through some imaages in test data with high resolution

In [ ]:
tmp = test_50.loc[test_50.pix == test_50.pix.max()]
print(tmp.head())
pname = list(tmp.pname)
print('\n',len(pname))

There are 14 images in test data with high resolution.

In [ ]:
image = Image.open(pname[0])
print(image.size)
display(image)

In [ ]:
image = Image.open(pname[1])
print(image.size)
display(image)

In [ ]:
image = Image.open(pname[13])
print(image.size)
display(image)

In [ ]:
image = Image.open(pname[12])
print(image.size)
display(image)

### There might be some relationship between test images with high resolution.
Let's display multiple images together

In [ ]:
fig=plt.figure(figsize=(12, 10))
columns = 4
rows = 3
for i in range(1, columns*rows +1):
    img = Image.open(pname[i])
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

### Lets go through some imaages in test data with low resolution

In [ ]:
tmp = test_50.loc[test_50.pix == test_50.pix.min()]
print(tmp.head())
pname = list(tmp.pname)
print('\n',len(pname))

Only one image in test data with low resolution.

In [ ]:
image = Image.open(pname[0])
print(image.size)
display(image)

This is the image with low resolution in test data.

In [ ]:
train_50.to_csv('landmark_train_facts.csv', index = False)
test_50.to_csv('landmark_test_facts.csv', index = False)

## Hope you will like this notebook. 


# Suggestions please...